Strategy is to first use ML to identify patterns in the data.

In [18]:
import speedml
import sys
# import os
# sys.path.append(os.getcwd())

sys.path.append(r'C:\Code\MFG598\final_project\production_modules')
from m1_2_data_ingestion import master_dataframe

In [19]:
master_dataframe.head()

,Complete Records
0,Complet...
1,Complete...
2,Complet...
3,Complete...
4,Complet...


In [20]:
sml = speedml.Speedml()
sml = sml.read_df(master_dataframe)

TypeError: Speedml.__init__() missing 3 required positional arguments: 'train', 'test', and 'target'